In [0]:
from pyspark.sql.types import StringType, IntegerType, ArrayType
import time
import pyspark.sql.functions as F
from pyspark.sql import Window
import requests
import json
import urllib.parse

#### Database Schema

<img src="https://i.ibb.co/hRqLR8d/Schema.png" alt="Schema" border="0">

In [0]:
raw_df = spark.read.format("delta").load("/user/hive/warehouse/scientific_publications")
display(raw_df.limit(5))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e3f5,null,null,null,null,null,null,null,List(),en,0,null,null,null,null,3GIO.,null,"List(null, null, null, null, null, null, null, null, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e133,"Drought is the first place in all the natural disasters in the world. It is especially serious in North China Plain. In this paper, different soil water content control levels at winter wheat growth stages are performed on Gucheng Ecological-Meteorological Integrated Observation Experiment Station of CAMS, China. Some canopy parameters, including growth conditions, dry weight, physiological parameters and hyperspectral reflectance, are measured from erecting stage to milk stage for winter wheat in 2009. The relationship between canopy parameters and soil relative moisture, canopy water content and water indices of winter wheat are established. The results show that some parameters, such as SPAD and dry weight of leaves, decrease with the increasing of soil relative moisture, while other parameters, including dry weight of caudexes, above ground dry weight, height, photosynthesis rate, intercellular CO 2 concentration, stomatal conductance and transpiration rate, increase corresponding to the soil relative moisture. Obvious linear relationship between stomatal conductance and transpiration rate is established with 45 samples, which R2 reaches to 0.6152. Finally, the fitting equations between canopy water content and water indices are regressed with b5, b6 and b7 of MODIS bands. The equations are best with b7 and worst with b5. So the fitting equations with b7 can be used to inverse the canopy water content of winter wheat using MODIS or other remote sensing images with similar bands range to MODIS in Hebei Province. © 2011 IEEE.","List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",,,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",,,,"List(handwriting recognition, prototypes, image segmentation, com

In [0]:
#Date, Keyword, Type, Publication, Venue, FOS, Organization, Author, Langue, AuthorRank
raw_df.columns

Out[103]: ['_id',
 'abstract',
 'authors',
 'doi',
 'fos',
 'isbn',
 'issn',
 'issue',
 'keywords',
 'lang',
 'n_citation',
 'page_end',
 'page_start',
 'pdf',
 'references',
 'title',
 'url',
 'venue',
 'volume',
 'year']

In [0]:
#Filter out titles that have less than 2 words
filtered_df = raw_df.withColumn("title_word_count", F.size(F.split("title", " "))).filter("title_word_count > 1")
# filtered_df = filtered_df.filter(F.col("doi").isNull())
# filtered_df = filtered_df.filter("n_citation > 3")
display(filtered_df.limit(200))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,title_word_count
53e99967b7602d97021a560a,"Ab initio predictions of secondary structures in proteins have to combine local predictions, based on short fragments of the protein sequence, with consistency restrictions, as not all locally plausible predictions may be simultaneously true. We use the fact that secondary structures are patterns of hydrogen bonds and that a single residue can participate in hydrogen bonds of at most one secondary structure. Consistency of fixed-sized pieces of secondary structures is the easiest to approximate and we formalize it as 1-2 matching problem. Consistency of entire secondary structures is a version of set packing. We also investigate how to form a simple problem if we add the requirement that the secondary structure and the loops that connect them fit together in a metric space. Every problem that we investigated is MAX-SNP hard and it has a constant factor approximation. Computational experience suggests that in biological instances, we can find nearly optimal solutions using heuristics.","List(List(53f45d3adabfaee0d9c0cb90, null, berman@cse.psu.edu, 5b86b6c2e1cd8e14a34ca959, Piotr Berman, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null), List(53f436cbdabfaedf43582e9f, null, null, 5b86b6c2e1cd8e14a34ca959, Jieun Jeong, null, null, null, null, The Pennsylvania State University, University Park, PA, USA, null, 5f71b2d91c455f439fe3eb4f, List(Penn State Univ, University Pk, PA 16802 USA), null, null))",10.1007/s00453-007-9068-8,"List(Algorithm, Heuristics, Set packing, Metric space, Ab initio, Hydrogen bond, Protein secondary structure, Mathematics, Protein structure, Theory of computation)",,0178-4617,1,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)",en,2,34,16,null,"List(53e9bdcdb7602d9704a7e6ae, 53e99d5eb7602d970261a1c4, 53e9bdcdb7602d9704a7e6ae, 53e99e3eb7602d970270422e, 53e9adffb7602d970380a762, 53e9b593b7602d97040d9071, 53e99c6fb7602d9702522122, 55a43e58612ca648688dc943, 53e99822b7602d97020431be)",Consistent Sets of Secondary Structures in Proteins,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",53,2009,7
53e99960b7602d97021a53fa,"In this work a new method of feature extraction for an interactive and adaptive recognizer for on-line handwritten alphanumeric characters has been proposed. The system is suitable for use in conjunction with magnetic pen based devices for inputting data to a data processing system or a computer terminal. The features are extracted from dynamically changing locations of the writing device. The new method of feature extraction is simple, computationally light and fast enough for adaptive on-line use. Extracted features are robust with respect to all possible distortions like shape, size, and orientation. For simulation experiment, numerals 0-9 are used. A single hidden layer feed forward neural network trained by Quickprop algorithm, a variation of error back propagation is used for recognition. Very high recognition rates, even for highly distorted samples have been achieved confirming high generalization capability of the extracted feature set.","List(List(53f36626dabfae4b3499aa7a, null, null, 5b86b9c0e1cd8e14a3625bbe, Basabi Chakraborty, null, null, null, null, Faculty of Software and Information Science, Iwate Prefectural University, 152-52 Aza Sugo, Takizawamura, Iwate 020-0193, Japan, null, 5f71b4211c455f439fe47d1e,

## Authors

In [0]:
def get_author_from_dblp(title, rank):
    URL = "http://dblp.org/search/publ/api?q=" + title.replace(" ", "+") + "&format=json"
    r = requests.get(url = URL)
    data = r.json()
    try:
        return data['result']['hits']['hit'][0]['info']['authors']['author'][rank-1]['text']
    except:
        return None
    
#get_author_from_dblpUDF = F.udf(lambda x, y:get_author_from_dblp(x, y), StringType()) 

In [0]:
authors = (filtered_df.limit(100).select("authors", "title", F.posexplode(F.col("authors")).alias("rank", "authors_exp"))
            .withColumn("rank", F.col("rank") + 1)
            .select("authors_exp.*","*")
            .select("_id", "rank", "name", "title")
            .withColumn("name", F.initcap(F.col("name")))
            )

authors2 = authors.filter(F.col("name").rlike(r"^\p{L}\.?(-\p{L}\.?)?\s.+"))
authors_rdd = authors2.rdd.map(lambda x: (x[0], x[1], x[2], x[3], get_author_from_dblp(x[3], x[1])))
authors2 = authors_rdd.toDF(["_id", "rank", "name", "title", "dblp_name"])
    
authors2 = (authors2.withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1)))

authors_raw = (authors.withColumn("dblp_name", F.col("name"))
           .union(authors2)
           .withColumn("last_name_raw", F.regexp_extract(F.col("name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1))
           .withColumn("dblp_last_name", F.regexp_extract(F.col("dblp_name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("name", F.when(F.col("last_name_raw") == F.col("dblp_last_name"), 
                                      F.col("dblp_name")).otherwise(F.col("name")))
           .withColumn("first_name", F.regexp_extract(F.col("name"), r"^(\S+)\s.+$", 1))
           .withColumn("last_name", F.regexp_extract(F.col("name"), r"^.+\s(\S+)$", 1))
           .withColumn("middle_name", F.regexp_extract(F.col("name"), r"^\S+\s(\S+)\s\S+$", 1))
          )

authors = (authors_raw.select("first_name", "last_name", "middle_name")
            .dropDuplicates()
            .withColumn("id", F.monotonically_increasing_id())
          )

authors = (authors_raw.join(authors, ["first_name", "last_name", "middle_name"])
           .select("id", "first_name", "last_name", "middle_name", "title", "rank")
           .dropDuplicates()
          )

display(authors)

id,first_name,last_name,middle_name,title,rank
0,Andreas,Herzig,,Dynamic Logic of Propositional Assignments: A Well-Behaved Variant of PDL,2
1,Fu-lai,Chung,,Stock time series pattern matching: Template-based vs. rule-based approaches,2
2,Ravi,Sundaram,,Secure and scalable match: overcoming the universal circuit bottleneck using group programs.,2
3,Qingtang,Liu,,Research on DRM-enabled learning objects model,1
4,Baruch,Mazor,,A trellis code construction and coding theorem for stationary Gaussian sources,1
5,Han-chin,Cheng,,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.,5
6,Weiwu,Shi,,Communication with Threads in Software DSMs,1
7,Peng,Liu,,In-broker access control: towards efficient end-to-end performance of information brokerage systems,3
8,I.,Prelog,,A distributed virtual reality-based system for neonatal decision-making training,3
9,Daniel,Pellicer,,Combinatorial Structure of Schulte's Chiral Polyhedra,1


In [0]:
#Old author transformations
"""
authors_test = (filtered_df.limit(100).select("authors", "title", F.posexplode(F.col("authors")).alias("rank", "authors_exp"))
           .withColumn("rank", F.col("rank") + 1)
           .select("authors_exp.*","*")
           .select("_id", "rank", "name", "title")
           .withColumn("name", F.initcap(F.col("name")))
           .withColumn("dblp_name", F.when(F.col("name").rlike(r"^\p{L}\.?(-\p{L}\.?)?\s.+"), 
                                            get_author_from_dblpUDF(F.col("title"), F.col("rank")))
                          .otherwise(F.col("name")))
           .withColumn("last_name_raw", F.regexp_extract(F.col("name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))
           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\D+)(\s\d\d\d\d)?$", 1))
           .withColumn("dblp_last_name", F.regexp_extract(F.col("dblp_name"), r"^.+\s(\S+)(\sJr\.)?$", 1))
           .withColumn("name", F.when(F.col("last_name_raw") == F.col("dblp_last_name"), 
                                      F.col("dblp_name")).otherwise(F.col("name")))
           .withColumn("first_name", F.regexp_extract(F.col("name"), r"^(\S+)\s.+$", 1))
           .withColumn("last_name", F.regexp_extract(F.col("name"), r"^.+\s(\S+)$", 1))
           .withColumn("middle_name", F.regexp_extract(F.col("name"), r"^\S+\s(\S+)\s\S+$", 1))
           .select("_id", "first_name", "last_name", "middle_name")
          )
          
display(authors_test)
"""

Out[107]: '\nauthors_test = (filtered_df.limit(100).select("authors", "title", F.posexplode(F.col("authors")).alias("rank", "authors_exp"))\n           .withColumn("rank", F.col("rank") + 1)\n           .select("authors_exp.*","*")\n           .select("_id", "rank", "name", "title")\n           .withColumn("name", F.initcap(F.col("name")))\n           .withColumn("dblp_name", F.when(F.col("name").rlike(r"^\\p{L}\\.?(-\\p{L}\\.?)?\\s.+"), \n                                            get_author_from_dblpUDF(F.col("title"), F.col("rank")))\n                          .otherwise(F.col("name")))\n           .withColumn("last_name_raw", F.regexp_extract(F.col("name"), r"^.+\\s(\\S+)(\\sJr\\.)?$", 1))\n           .withColumn("dblp_name", F.when(F.col("dblp_name") == "null", F.col("name")).otherwise(F.col("dblp_name")))\n           .withColumn("dblp_name", F.regexp_extract(F.col("dblp_name"), r"^(\\D+)(\\s\\d\\d\\d\\d)?$", 1))\n           .withColumn("dblp_last_name", F.regexp_extract(F.col("d

In [0]:
res = get_author_from_dblp("Global Approach for Compiled Bit-True Simulation of DSP Systems", 4)
print(res)

J. A. Peperstraete


## Organizations

In [0]:
def get_organization(name, country):
    try:
        URL = "https://api.ror.org/organizations?query=" + name.replace(" ", "+")
        r = requests.get(url = URL)
        data = r.json()
        if data['number_of_results'] == 0:
            return "No results"
        for i in range(data['number_of_results']):
            found_country = data['items'][i]["country"]["country_name"]
            if country == "" or country == found_country:
                return data['items'][i]["name"]+";"+data['items'][i]["addresses"][0]["city"]+";"+found_country
    except:
        return "No results"
get_organizationUDF = F.udf(lambda x:get_organization(x), StringType()) 

In [0]:
countries = "(,|\s)(Afghanistan|Albania|Algeria|Andorra|Angola|Antigua and Barbuda|Argentina|Armenia|Australia|Austria|Azerbaijan|Bahamas|Bahrain|Bangladesh|Barbados|Belarus|Belgium|Belize|Benin|Bhutan|Bolivia|Bosnia and Herzegovina|Botswana|Brazil|Brunei|Bulgaria|Burkina Faso|Burundi|Cabo Verde|Cambodia|Cameroon|Canada|Central African Republic|Chad|Chile|China|Colombia|Comoros|Democratic Republic of the Congo|Republic of the Congo|Costa Rica|Cote d'Ivoire|Croatia|Cuba|Cyprus|Czech Republic|Denmark|Djibouti|Dominica|Dominican Republic|Ecuador|Egypt|El Salvador|Equatorial Guinea|Eritrea|Estonia|Ethiopia|Fiji|Finland|France|Gabon|Gambia|Georgia|Germany|Ghana|Greece|Grenada|Guatemala|Guinea|Guinea-Bissau|Guyana|Haiti|Honduras|Hungary|Iceland|India|Indonesia|Iran|Iraq|Ireland|Israel|Italy|Jamaica|Japan|Jordan|Kazakhstan|Kenya|Kiribati|Kosovo|Kuwait|Kyrgyzstan|Laos|Latvia|Lebanon|Lesotho|Liberia|Libya|Liechtenstein|Lithuania|Luxembourg|North Macedonia|Madagascar|Malawi|Malaysia|Maldives|Mali|Malta|Marshall Islands|Mauritania|Mauritius|Mexico|Micronesia|Moldova|Monaco|Mongolia|Montenegro|Morocco|Mozambique|Myanmar|Namibia|Nauru|Nepal|Netherlands|New Zealand|Nicaragua|Niger|Nigeria|North Korea|Norway|Oman|Pakistan|Palau|Palestine|Panama|Papua New Guinea|Paraguay|Peru|Philippines|Poland|Portugal|Qatar|Romania|Russia|Rwanda|Saint Kitts and Nevis|Saint Lucia|Saint Vincent and the Grenadines|Samoa|San Marino|Sao Tome and Principe|Saudi Arabia|Senegal|Serbia|Seychelles|Sierra Leone|Singapore|Slovakia|Slovenia|Solomon Islands|Somalia|South Africa|South Korea|South Sudan|Spain|Sri Lanka|Sudan|Suriname|Swaziland|Sweden|Switzerland|Syria|Taiwan|Tajikistan|Tanzania|Thailand|Timor-Leste|Togo|Tonga|Trinidad and Tobago|Tunisia|Turkey|Turkmenistan|Tuvalu|Uganda|Ukraine|United Arab Emirates|UAE|United Kingdom|UK|United States of America|USA|United States|US|Uruguay|Uzbekistan|Vanuatu|Vatican City|Venezuela|Vietnam|Yemen|Zambia|Zimbabwe)(,|\s|$)"


organization = (filtered_df.limit(100).select("authors", "title", F.explode(F.col("authors")).alias("authors_exp"))
               .select("authors_exp.*","*")
               .withColumn("author", F.col("name"))
               .select("author", "org")
               .filter(F.col("org").isNotNull())
               .dropDuplicates()
                # ., +, *, ?, ^, $, (, ), [, ], {, }, |, \
               .withColumn("strip_org", F.regexp_replace(F.col("org"), r'[\+-=#&\|><!\(\)\{\}\[\]\^"~\*\?:\\/]', " "))
               .withColumn("county", F.regexp_extract(F.col("org"), countries, 2))
               .withColumn("county", F.regexp_replace("county", "United States of America|USA|US", "United States"))
               .withColumn("county", F.regexp_replace("county", "UK", "United Kingdom"))
               .withColumn("county", F.regexp_replace("county", "UAE", "United Arab Emirates"))
               )

organization_rdd = organization.rdd.map(lambda x: (x[0], x[1], x[2], x[3], get_organization(x[2], x[3])))
organization_raw = (organization_rdd.toDF(["author", "org", "strip_org", "country", "api_org"])
                    .withColumn("api_name", F.regexp_extract(F.col("api_org"), r"^(.+);.+;.+$", 1))
                    .withColumn("match", F.expr(r"regexp_extract(org, concat('(', api_name,')'), 0)"))
                    .withColumn("name", F.when(F.col("match") == "", F.col("org"))
                               .otherwise(F.col("api_name")))
                    .withColumn("city", F.when(F.col("match") == "", "")
                               .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;(.+);.+$", 1)))
                    .withColumn("country", F.when(F.col("match") == "", F.col("country"))
                               .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;.+;(.+)$", 1)))
                   )

organization = (organization_raw.select("name", "city", "country")
                .dropDuplicates()
                .withColumn("id", F.monotonically_increasing_id())
               )

organization = organization_raw.join(organization, ["name", "city", "country"]).select("id", "name", "city", "country", "author")

display(organization)

id,name,city,country,author
0,"Tokyo Inst. Tech., Yokohama, Japan",,Japan,Takao Terano
1,Georgia Institute of Technology,Atlanta,United States,Farshid Delgosha
1,Georgia Institute of Technology,Atlanta,United States,Erman Ayday
1,Georgia Institute of Technology,Atlanta,United States,Faramarz Fekri
2,"Universit ¨ , at Karlsruhe",,,Gerhard Starke
3,National University of Singapore,Singapore,Singapore,Yong Zeng
3,National University of Singapore,Singapore,Singapore,Zhifeng Bao
3,National University of Singapore,Singapore,Singapore,Tok Wang Ling
3,National University of Singapore,Singapore,Singapore,Luochen Li
4,York University Toronto Ontario M3J 1P3 Canada,,Canada,Daniel Pellicer


In [0]:
#Old organization transformations
"""
organization = (filtered_df.limit(1000).select("authors", "title", F.explode(F.col("authors")).alias("authors_exp"))
               .select("authors_exp.*","*")
               .select("org")
               .filter(F.col("org").isNotNull())
               .dropDuplicates()
                # ., +, *, ?, ^, $, (, ), [, ], {, }, |, \
               .withColumn("strip_org", F.regexp_replace(F.col("org"), r'[\+-=#&\|><!\(\)\{\}\[\]\^"~\*\?:\\/]', " "))
               .withColumn("api_org", get_organizationUDF(F.col("strip_org")))
               .withColumn("name", F.when(F.col("api_org") == "No results", F.col("org"))
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^(.+);.+;.+$", 1)))
               .withColumn("city", F.when(F.col("api_org") == "No results", "")
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;(.+);.+$", 1)))
               .withColumn("country", F.when(F.col("api_org") == "No results", "")
                           .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;.+;(.+)$", 1)))
               .select("name", "city", "country")
               )

display(organization)
"""

Out[111]: '\norganization = (filtered_df.limit(1000).select("authors", "title", F.explode(F.col("authors")).alias("authors_exp"))\n               .select("authors_exp.*","*")\n               .select("org")\n               .filter(F.col("org").isNotNull())\n               .dropDuplicates()\n                # ., +, *, ?, ^, $, (, ), [, ], {, }, |,                .withColumn("strip_org", F.regexp_replace(F.col("org"), r\'[\\+-=#&\\|><!\\(\\)\\{\\}\\[\\]\\^"~\\*\\?:\\/]\', " "))\n               .withColumn("api_org", get_organizationUDF(F.col("strip_org")))\n               .withColumn("name", F.when(F.col("api_org") == "No results", F.col("org"))\n                           .otherwise(F.regexp_extract(F.col("api_org"), r"^(.+);.+;.+$", 1)))\n               .withColumn("city", F.when(F.col("api_org") == "No results", "")\n                           .otherwise(F.regexp_extract(F.col("api_org"), r"^.+;(.+);.+$", 1)))\n               .withColumn("country", F.when(F.col("api_org") == "No result

In [0]:
def get_organization2(name):
    URL = "https://api.ror.org/organizations?query=" + urllib.parse.quote(name)
    r = requests.get(url = URL)
    data = r.json()
    if data['number_of_results'] == 0:
        return "No results"
    return data['items'][0]["name"]+";"+data['items'][0]["addresses"][0]["city"]+";"+data['items'][0]["country"]["country_name"]

res = get_organization2("SEM, Tsinghua University, Beijing, China")

print(res)

Tsinghua Holdings (China);Beijing;China


## Publications

In [0]:
publication_raw = filtered_df.limit(100).select("_id", "title", "volume", "n_citation", "doi", "url")

display(publication_raw)

_id,title,volume,n_citation,doi,url
53e99967b7602d97021a560a,Consistent Sets of Secondary Structures in Proteins,53,2,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)"
53e99960b7602d97021a53fa,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters,148,19,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)"
53e99960b7602d97021a5416,King John Divided,19,0,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181)
53e99960b7602d97021a541f,Peer Assist Live Streaming Overlay for Next-Generation-Networks,1,0,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)"
53e99960b7602d97021a5427,Paramecium: Assembling Raw Nodes into Composite Cells,3222,2,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)"
53e99967b7602d97021a5645,Radar Signal Retrodiffusion by Water Surface,2,1,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)"
53e99960b7602d97021a543e,Custos Remote on-demand healthcare aided with wireless sensors and mobile phones.,,5,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)"
53e99967b7602d97021a566b,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation,,13,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314)
53e99960b7602d97021a5451,Weighted census transform for feature representation,,2,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)"
53e99967b7602d97021a5814,A signaling and control architecture for mobility support in wireless ATM networks,1,134,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)"


In [0]:
def check_return_data(check_type, data, cur):
    if data is not None and check_type in data:
        return data[check_type]
    else:
        return cur
    
def update_df(df, doi_list, data_list, data_str, is_int=False):
    data_dict = dict(zip(doi_list, data_list))
    update_data = udf(lambda x: data_dict[x], IntegerType() if is_int else StringType())
    return df.withColumn(data_str, update_data(F.col('doi')))

def get_publication_data(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    url_list = df.select(F.col("url")).rdd.flatMap(lambda x: x).collect()
    title_list = df.select(F.col("title")).rdd.flatMap(lambda x: x).collect()
    citation_list = df.select(F.col("n_citation")).rdd.flatMap(lambda x: x).collect()
    volume_list = df.select(F.col("volume")).rdd.flatMap(lambda x: x).collect()
    new_volume_list = []
    n_citation_list = []
    series_list = []
    new_doi_list = []
    for i, doi in enumerate(doi_list):
        try:
            if doi == "" or doi is None:
                if "doi" in url_list[i][0]:
                    doi_req = url_list[i][0].split("org/")[-1]
                else:
                    raise Exception
            else:
                doi_req = doi
            
            new_doi_list.append(doi_req)
            response = requests.get(f"https://api.crossref.org/works/{doi_req}")
            data = response.json()['message']
        except Exception:
            new_doi_list.append(None)
            data = None

        n_citation_list.append(int(check_return_data('is-referenced-by-count', data, citation_list[i])))
        new_volume_list.append(check_return_data('volume', data, volume_list[i]))
        temp = check_return_data('container-title', data, None)
        series_list.append(temp if temp == None else temp[0])

        time.sleep(0.05)
    
    df = update_df(df, doi_list, n_citation_list, 'n_citation')
    df = update_df(df, doi_list, new_volume_list, 'volume')
    df = update_df(df, doi_list, series_list, 'series')
    df = update_df(df, doi_list, new_doi_list, 'doi')
    
    return df

In [0]:
publication = get_publication_data(publication_raw).filter("n_citation > 2")
# publication = get_publication_dataget_crossref_data(publication_raw).select("_id", "title", "volume", "series", "n_citation").filter("n_citation > 2")
display(publication)

_id,title,volume,n_citation,doi,url,series
53e99960b7602d97021a53fa,A new feature extraction technique for on-line recognition of handwritten alphanumeric characters,148,22,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)",Information Sciences
53e99967b7602d97021a566b,Group Elevator Scheduling with Advanced Traffic Information for Normal Operations and Coordinated Emergency Evacuation,,10,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),Proceedings of the 2005 IEEE International Conference on Robotics and Automation
53e99967b7602d97021a5814,A signaling and control architecture for mobility support in wireless ATM networks,1,23,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)",Mobile Networks and Applications
53e99967b7602d97021a581e,DBDOC: querying and browsing databases and interrelated documents,,4,10.1145/1557670.1557688,"List(http://dx.doi.org/10.1145/1557670.1557688, http://doi.acm.org/10.1145/1557670.1557688)",Proceedings of the First International Workshop on Keyword Search on Structured Data
53e99967b7602d97021a5680,Improving Learner Performance with Data Sampling and Boosting,1,3,10.1109/ICTAI.2008.58,"List(http://dx.doi.org/10.1109/ICTAI.2008.58, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4669723)",2008 20th IEEE International Conference on Tools with Artificial Intelligence
53e99960b7602d97021a546b,To use PYA tool or not for learning Japanese hand alphabets,,3,10.1145/1178823.1178884,"List(http://dx.doi.org/10.1145/1178823.1178884, http://doi.acm.org/10.1145/1178823.1178884)",Proceedings of the 2006 ACM SIGCHI international conference on Advances in computer entertainment technology
53e99960b7602d97021a546d,On boolean functions with generalized cryptographic properties,2004,12,10.1007/978-3-540-30556-9_11,"List(http://dx.doi.org/10.1007/978-3-540-30556-9_11, http://eprint.iacr.org/2004/259, http://www.webofknowledge.com/)",Progress in Cryptology - INDOCRYPT 2004
53e99960b7602d97021a5478,Comparative Study of Sequential Pattern Mining Models,6,10,10.1007/11498186_3,"List(http://dx.doi.org/10.1007/11498186_3, http://www.webofknowledge.com/)",null
53e99967b7602d97021a5847,User profiling with Case-Based Reasoning and Bayesian Networks,,14,10.1155/2009/921324,List(),null
53e99967b7602d97021a584a,"TOOLS 1999: 31st International Conference on Technology of Object-Oriented Languages and Systems, 22-25 September 1999, Nanjing, China",,14,10.1155/2009/921324,null,null


## Types

In [0]:
types_raw = filtered_df.limit(100).select("_id", "volume", "issue", "doi", "url", "venue")

display(types_raw)

_id,volume,issue,doi,url,venue
53e99967b7602d97021a560a,53,1,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)"
53e99960b7602d97021a53fa,148,1-4,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)","List(555036b77cea80f95414b7e3, null, null, null, null, null, null, Inf. Sci., null, null, null, null, 0)"
53e99960b7602d97021a5416,19,2,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181),"List(555036d37cea80f95415b0b3, null, null, null, null, null, null, LLC, null, null, null, null, 0)"
53e99960b7602d97021a541f,1,4,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)","List(555036cc7cea80f954158149, null, null, International Journal of Handheld Computing Research, null, null, null, IJHCR, null, null, null, null, 0)"
53e99960b7602d97021a5427,3222,,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)","List(null, 0302-9743, null, null, null, 1611-3349, null, LECTURE NOTES IN COMPUTER SCIENCE, null, LECTURE NOTES IN COMPUTER SCIENCE, null, J, null)"
53e99967b7602d97021a5645,2,,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)","List(53a7297c20f7420be8bd4ae5, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)"
53e99960b7602d97021a543e,,,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)","List(555037277cea80f954176d3b, null, null, Systems, Man and Cybernetics, null, null, null, SMC, null, null, null, null, 0)"
53e99967b7602d97021a566b,,,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),"List(53a726bb20f7420be8b7f846, null, null, International Conference on Robotics and Automation, null, null, null, ICRA, null, null, null, null, 0)"
53e99960b7602d97021a5451,,,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)","List(555037017cea80f95416d334, null, null, International Conference on Ubiquitous Robots and Ambient Intelligence, null, null, null, URAI, null, null, null, null, 0)"
53e99967b7602d97021a5814,1,3,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)","List(53a730a020f7420be8cff2d8, null, null, International Conference on Communications, null, null, null, MONET, null, null, null, null, 0)"


In [0]:
types_pre_api = types_raw.withColumn("type", F.when(F.col("venue.raw").contains("@"), "workshop")
                   .when(((F.col("volume").isNotNull()) & (F.col("volume") != "")) | ((F.col("issue").isNotNull()) & (F.col("issue") != "")), "journal-article")
                   .otherwise("conference paper"))

display(types_pre_api)

_id,volume,issue,doi,url,venue,type
53e99967b7602d97021a560a,53,1,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",journal-article
53e99960b7602d97021a53fa,148,1-4,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)","List(555036b77cea80f95414b7e3, null, null, null, null, null, null, Inf. Sci., null, null, null, null, 0)",journal-article
53e99960b7602d97021a5416,19,2,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181),"List(555036d37cea80f95415b0b3, null, null, null, null, null, null, LLC, null, null, null, null, 0)",journal-article
53e99960b7602d97021a541f,1,4,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)","List(555036cc7cea80f954158149, null, null, International Journal of Handheld Computing Research, null, null, null, IJHCR, null, null, null, null, 0)",journal-article
53e99960b7602d97021a5427,3222,,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)","List(null, 0302-9743, null, null, null, 1611-3349, null, LECTURE NOTES IN COMPUTER SCIENCE, null, LECTURE NOTES IN COMPUTER SCIENCE, null, J, null)",journal-article
53e99967b7602d97021a5645,2,,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)","List(53a7297c20f7420be8bd4ae5, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",journal-article
53e99960b7602d97021a543e,,,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)","List(555037277cea80f954176d3b, null, null, Systems, Man and Cybernetics, null, null, null, SMC, null, null, null, null, 0)",conference paper
53e99967b7602d97021a566b,,,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),"List(53a726bb20f7420be8b7f846, null, null, International Conference on Robotics and Automation, null, null, null, ICRA, null, null, null, null, 0)",conference paper
53e99960b7602d97021a5451,,,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)","List(555037017cea80f95416d334, null, null, International Conference on Ubiquitous Robots and Ambient Intelligence, null, null, null, URAI, null, null, null, null, 0)",conference paper
53e99967b7602d97021a5814,1,3,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)","List(53a730a020f7420be8cff2d8, null, null, International Conference on Communications, null, null, null, MONET, null, null, null, null, 0)",journal-article


In [0]:
def check_return_data(check_type, data, cur):
    if data is not None and check_type in data:
        return data[check_type]
    else:
        return cur
    
def update_df(df, doi_list, data_list, data_str, is_int=False):
    data_dict = dict(zip(doi_list, data_list))
    update_data = udf(lambda x: data_dict[x], IntegerType() if is_int else StringType())
    return df.withColumn(data_str, update_data(F.col('doi')))

def get_type_data(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    url_list = df.select(F.col("url")).rdd.flatMap(lambda x: x).collect()
    type_list = df.select(F.col("type")).rdd.flatMap(lambda x: x).collect()
    new_type_list = []
    for i, doi in enumerate(doi_list):
        try:
            if doi == "" or doi is None:
                if "doi" in url_list[i][0]:
                    doi_req = url_list[i][0].split("org/")[-1]
                else:
                    raise Exception
            else:
                doi_req = doi
            
            response = requests.get(f"https://api.crossref.org/works/{doi_req}")
            data = response.json()['message']
        except Exception:
            data = None

        new_type_list.append(check_return_data('type', data, type_list[i]))

        time.sleep(0.05)

    df = update_df(df, doi_list, new_type_list, 'type')
    
    return df

In [0]:
types = get_type_data(types_pre_api)

display(types)

_id,volume,issue,doi,url,venue,type
53e99967b7602d97021a560a,53,1,10.1007/s00453-007-9068-8,"List(http://dx.doi.org/10.1007/s00453-007-9068-8, http://dx.doi.org/https://doi.org/10.1007/s00453-007-9068-8, https://link.springer.com/article/10.1007/s00453-007-9068-8, https://dl.acm.org/citation.cfm?id=3118965&picked=prox&preflayout=flat, http://www.webofknowledge.com/)","List(539078f220f770854f5a882f, null, null, null, null, null, null, Algorithmica, null, null, null, null, 0)",journal-article
53e99960b7602d97021a53fa,148,1-4,10.1016/S0020-0255(02)00276-1,"List(http://dx.doi.org/10.1016/S0020-0255(02)00276-1, https://www.sciencedirect.com/science/article/pii/S0020025502002761)","List(555036b77cea80f95414b7e3, null, null, null, null, null, null, Inf. Sci., null, null, null, null, 0)",journal-article
53e99960b7602d97021a5416,19,2,10.1093/llc/19.2.181,List(http://dx.doi.org/10.1093/llc/19.2.181),"List(555036d37cea80f95415b0b3, null, null, null, null, null, null, LLC, null, null, null, null, 0)",journal-article
53e99960b7602d97021a541f,1,4,10.4018/jhcr.2010100102,"List(http://dx.doi.org/10.4018/jhcr.2010100102, http://dx.doi.org/10.4018/978-1-4666-0921-1.ch017)","List(555036cc7cea80f954158149, null, null, International Journal of Handheld Computing Research, null, null, null, IJHCR, null, null, null, null, 0)",journal-article
53e99960b7602d97021a5427,3222,,10.1007/978-3-540-30141-7_67,"List(http://dx.doi.org/10.1007/978-3-540-30141-7_67, http://www.webofknowledge.com/)","List(null, 0302-9743, null, null, null, 1611-3349, null, LECTURE NOTES IN COMPUTER SCIENCE, null, LECTURE NOTES IN COMPUTER SCIENCE, null, J, null)",book-chapter
53e99967b7602d97021a5645,2,,10.1109/IGARSS.2009.5418122,"List(http://dx.doi.org/10.1109/IGARSS.2009.5418122, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=5418122)","List(53a7297c20f7420be8bd4ae5, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",proceedings-article
53e99960b7602d97021a543e,,,10.1109/ICSMC.2008.4811630,"List(http://dx.doi.org/10.1109/ICSMC.2008.4811630, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=4811630)","List(555037277cea80f954176d3b, null, null, Systems, Man and Cybernetics, null, null, null, SMC, null, null, null, null, 0)",proceedings-article
53e99967b7602d97021a566b,,,10.1109/ROBOT.2005.1570314,List(http://dx.doi.org/10.1109/ROBOT.2005.1570314),"List(53a726bb20f7420be8b7f846, null, null, International Conference on Robotics and Automation, null, null, null, ICRA, null, null, null, null, 0)",proceedings-article
53e99960b7602d97021a5451,,,10.1109/URAI.2013.6677409,"List(http://dx.doi.org/10.1109/URAI.2013.6677409, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=6677409)","List(555037017cea80f95416d334, null, null, International Conference on Ubiquitous Robots and Ambient Intelligence, null, null, null, URAI, null, null, null, null, 0)",proceedings-article
53e99967b7602d97021a5814,1,3,10.1007/BF01193262,"List(http://dx.doi.org/10.1007/BF01193262, http://dx.doi.org/10.1109/ICC.1996.542243, http://dx.doi.org/https://doi.org/10.1007/BF01193262, https://link.springer.com/article/10.1007/BF01193262)","List(53a730a020f7420be8cff2d8, null, null, International Conference on Communications, null, null, null, MONET, null, null, null, null, 0)",journal-article


## Keywords

In [0]:
keywords_raw = filtered_df.limit(100).select("_id", "keywords")

display(keywords_raw)

_id,keywords
53e99967b7602d97021a560a,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)"
53e99960b7602d97021a53fa,"List(adaptive on-line use, data processing system, feature extraction, inputting data, on-line recognition, handwritten alphanumeric character, feature set, new feature extraction technique, extracted feature, adaptive recognizer, new method, high generalization capability, high recognition rate, feed forward neural network, error back propagation, simulation experiment, data processing, artificial neural network)"
53e99960b7602d97021a5416,List()
53e99960b7602d97021a541f,"List(scalability, ims, p2p, ngn, next generation network)"
53e99960b7602d97021a5427,List()
53e99967b7602d97021a5645,"List(airborne radar, backscatter, height measurement, oceanographic regions, oceanographic techniques, remote sensing by radar, synthetic aperture radar, Camargue region, DRIVE radar, France, ONERA BUSARD platform, SAR instrument, backscattering coefficient profiles, dry surfaces, future space Ka band altimetry missions, nadir backscattering, pond surfaces, sea surfaces, sensor configuration, specular behavior, steep radar incident angle, synthetic aperture radar, water surface radar signal retrodiffusion, wet surfaces, Ka band, altimetry mode, radar sensor)"
53e99960b7602d97021a543e,"List(Java, Web services, biomedical telemetry, health care, mobile computing, sensor fusion, telemedicine, wireless sensor networks, Custos, ICT technique, Internet, Squawk Java J2ME, Sun SPOT, UPnP digital-home platform, Web service, Wei-Gong memorial hospital, clinical trial, geriatric psychiatry ward, mobile phone, multisensor data fusion, remote on-demand healthcare, small programmable object technology, smart phone, wireless sensor network, Smart Environments, Telemedicine, Telemonitoring, Wireless Sensor Networks)"
53e99967b7602d97021a566b,"List(information technology, simulation, transportation, objective function, dynamic programming, state space, sensor network, elevators, uncertainty, testing, normal operator, look ahead, lagrangian relaxation, local search)"
53e99960b7602d97021a5451,"List(weighted census transform, visual feature representation method, image representation, binary code bit, face recognition, ar face datasets, modified census transform, wct, statistical analysis, feature representation model, visual image classification, feature representation, training dataset, compressed sensing, image classification, statistical approach, mct, entropy information, transforms, entropy, compressive sensing technique, pattern classification)"
53e99967b7602d97021a5814,"List(Wireless Link, Mobile Terminal, Control Protocol, Control Architecture, Control Layer)"


In [0]:
keywords = keywords_raw.filter(F.size(keywords_raw.keywords) > 0)

display(keywords)

_id,keywords
53e99967b7602d97021a560a,"List(Secondary Structure, Maximum Weight, Local Prediction, Horizontal Edge, Connector Edge)"
53e99960b7602d97021a53fa,"List(adaptive on-line use, data processing system, feature extraction, inputting data, on-line recognition, handwritten alphanumeric character, feature set, new feature extraction technique, extracted feature, adaptive recognizer, new method, high generalization capability, high recognition rate, feed forward neural network, error back propagation, simulation experiment, data processing, artificial neural network)"
53e99960b7602d97021a541f,"List(scalability, ims, p2p, ngn, next generation network)"
53e99967b7602d97021a5645,"List(airborne radar, backscatter, height measurement, oceanographic regions, oceanographic techniques, remote sensing by radar, synthetic aperture radar, Camargue region, DRIVE radar, France, ONERA BUSARD platform, SAR instrument, backscattering coefficient profiles, dry surfaces, future space Ka band altimetry missions, nadir backscattering, pond surfaces, sea surfaces, sensor configuration, specular behavior, steep radar incident angle, synthetic aperture radar, water surface radar signal retrodiffusion, wet surfaces, Ka band, altimetry mode, radar sensor)"
53e99960b7602d97021a543e,"List(Java, Web services, biomedical telemetry, health care, mobile computing, sensor fusion, telemedicine, wireless sensor networks, Custos, ICT technique, Internet, Squawk Java J2ME, Sun SPOT, UPnP digital-home platform, Web service, Wei-Gong memorial hospital, clinical trial, geriatric psychiatry ward, mobile phone, multisensor data fusion, remote on-demand healthcare, small programmable object technology, smart phone, wireless sensor network, Smart Environments, Telemedicine, Telemonitoring, Wireless Sensor Networks)"
53e99967b7602d97021a566b,"List(information technology, simulation, transportation, objective function, dynamic programming, state space, sensor network, elevators, uncertainty, testing, normal operator, look ahead, lagrangian relaxation, local search)"
53e99960b7602d97021a5451,"List(weighted census transform, visual feature representation method, image representation, binary code bit, face recognition, ar face datasets, modified census transform, wct, statistical analysis, feature representation model, visual image classification, feature representation, training dataset, compressed sensing, image classification, statistical approach, mct, entropy information, transforms, entropy, compressive sensing technique, pattern classification)"
53e99967b7602d97021a5814,"List(Wireless Link, Mobile Terminal, Control Protocol, Control Architecture, Control Layer)"
53e99967b7602d97021a581e,"List(user-defined functions, typical database schema, document-to-metadata link, different level, interrelated document, typical information retrieval capability, sql code generation, browsing databases, large collection, acceptable performance, basic set, information retrieval, xml, code generation)"
53e99967b7602d97021a5680,"List(data mining, learning (artificial intelligence), boosting, data mining, data sampling techniques, learning)"


### Trying to install spacy
Getting error

In [0]:
%sh
pip install spacy
python -m spacy download en_core_web_sm

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-dd82c345-ace1-4cec-8396-84de9411dfe4
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dd82c345-ace1-4cec-8396-84de9411dfe4/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dd82c345-ace1-4cec-8396-84de9411dfe4/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def is_city_location_person(keyword):
    # Process the keyword using the spaCy NER model
    doc = nlp(keyword)
    
    # Check if any of the entities recognized by the model are of type GPE (city or location) or PERSON
    for ent in doc.ents:
        if ent.label_ in ['GPE', 'PERSON']:
            return True
    
    return False

def update_df(df, doi_list, data_list, data_str, is_int=False):
    data_dict = dict(zip(doi_list, data_list))
    update_data = udf(lambda x: data_dict[x], IntegerType() if is_int else StringType())
    return df.withColumn(data_str, update_data(F.col('doi')))

def get_type_data(df):
    doi_list = df.select(F.col("doi")).rdd.flatMap(lambda x: x).collect()
    keywords_list = df.select(F.col("keywords")).rdd.flatMap(lambda x: x).collect()
    new_keywords_list = []
    for i, keys in enumerate(keywords_list):
        temp = []
        for k in key:
            if not is_city_location_person(k):
                temp.append(k)
        new_keywords_list.append(temp)
        
    df = update_df(df, doi_list, new_type_list, 'keywords')
    
    return df

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
<command-1103867142204479> in <cell line: 1>()
----> 1 import spacy
      2 
      3 # Load the spaCy model
      4 nlp = spacy.load("en_core_web_sm")
      5 

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

/local_disk0/.ephemeral_nfs/envs/pythonEnv-dd82c345-ace1-4cec-8396-84de9411dfe4/lib/python3.9/site-packages/spacy/__init__.py in <module>
      4 
      5 # set library-specific cust

# DATE

In [0]:
# https://pypi.org/project/habanero/
from habanero import Crossref
cr = Crossref()

def getDateHabanero(doi, year):
    formated = [None, None, None]
    try:
        result = cr.works(ids = doi)['message']['issued']['date-parts'][0]
        for i in [0, 1, 2]:
            formated[i] = result[i]
    except:
        pass
    if formated[0] == None:
        formated[0] = year 
    return formated

getDateHabaneroUDF = udf(getDateHabanero, ArrayType(StringType()))

In [0]:
dates_df = (filtered_df
            #.limit(100)
            #.select('_id', "year", "doi")
            .withColumn("Date", getDateHabaneroUDF(F.col("doi"), F.col("year")))
            .select('_id',
                    F.col("Date")[2].alias("Day"),
                    F.col("Date")[1].alias("Month"),
                    F.col("Date")[0].alias("Year")
                   )
           )

display(dates_df)

_id,Day,Month,Year
53e99967b7602d97021a560a,10,11,2007
53e99960b7602d97021a53fa,null,12,2002
53e99960b7602d97021a5416,1,6,2004
53e99960b7602d97021a541f,1,10,2010
53e99960b7602d97021a5427,null,null,2004
53e99967b7602d97021a5645,null,null,2009
53e99960b7602d97021a543e,null,10,2008
53e99967b7602d97021a566b,null,null,2005
53e99960b7602d97021a5451,null,10,2013
53e99967b7602d97021a5814,null,10,1996


# LANGUAGE

In [0]:
display(raw_df.groupBy("lang").count())
display(filtered_df.groupBy("lang").count())

lang,count
en,249984
zh,16


lang,count
en,244930


In [0]:
lang_df = filtered_df.select("_id", "lang")
display(lang_df)

_id,lang
53e99967b7602d97021a560a,en
53e99960b7602d97021a53fa,en
53e99960b7602d97021a5416,en
53e99960b7602d97021a541f,en
53e99960b7602d97021a5427,en
53e99967b7602d97021a5645,en
53e99960b7602d97021a543e,en
53e99967b7602d97021a566b,en
53e99960b7602d97021a5451,en
53e99967b7602d97021a5814,en
